# ItalianSignNet study

### Importing Required Libraries

In [ ]:
!pip cache purge
!pip install -q tensorflow-model-optimization
!pip install tf-keras
!pip install lime scikit-image

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_model_optimization as tfmot
#import tf_keras as keras
import tensorflow.keras as keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

np.random.seed(42)

### Boiterplate

In [ ]:
data_dir = '/kaggle/input/dataset-2025-04-23-eps-100-changed/dataset_20250423_200322_eps_100_changed/'
train_path = f'{data_dir}/train'
test_path = f'{data_dir}/test'
val_set = f'{data_dir}/validation'

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3

NUM_CATEGORIES = len(os.listdir(train_path))
lr = 0.001
epochs = 30

classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

### Visualizing The Dataset

In [ ]:
folders = os.listdir(train_path)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_path + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])
    
# Sorting the dataset on the basis of number of images in each class
zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [ list(tuple) for tuple in  tuples]

# Plotting the number of images in each class
plt.figure(figsize=(10,5))  
plt.bar(class_num, train_number)
plt.xticks(class_num, rotation='vertical')
plt.show()

## Collecting the Training Data

In [ ]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/train/' + f"{i:02d}"

    # Skip folder of label that not exists 
    if not os.path.isdir(path):
        continue
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(f"{i:02d}")
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

### Shuffling the training data

In [ ]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

## Collecting validation set

In [ ]:
X_train = image_data.astype('float32') / 255.

val_data = []
val_labels = []
for label_name in os.listdir(val_set):
    label_dir = os.path.join(val_set, label_name)
    if not os.path.isdir(label_dir):
        continue
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = cv2.imread(img_path)
        img = Image.fromarray(img, 'RGB').resize((IMG_HEIGHT, IMG_WIDTH))
        val_data.append(np.array(img))
        val_labels.append(int(label_name))

X_val = np.array(val_data, dtype='float32') / 255.

print("X_train.shape", X_train.shape)
print("X_val.shape",   X_val.shape)

## One hot encoding the labels

In [ ]:
y_train = keras.utils.to_categorical(image_labels, NUM_CATEGORIES)
val_labels = np.array(val_labels) 
y_val = keras.utils.to_categorical(val_labels, NUM_CATEGORIES)
print("y_train.shape", y_train.shape)
print("y_val.shape",   y_val.shape)

## Making the model

In [ ]:
model = keras.models.Sequential([ 
    keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),
    keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])
model.summary()

In [ ]:
def improved_model(input_shape=(IMG_HEIGHT,IMG_WIDTH,channels), num_classes=43):
    inputs = keras.Input(shape=input_shape)
    x = keras.layers.SeparableConv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = keras.layers.BatchNormalization()(x)
    # Blocco residual 1
    res = x
    x = keras.layers.SeparableConv2D(32,(3,3),activation='relu',padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Add()([x, res])
    x = keras.layers.MaxPool2D()(x)

    # Blocco residual 2
    res = x
    x = keras.layers.SeparableConv2D(32,(3,3),activation='relu',padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.SeparableConv2D(32,(3,3),activation='relu',padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Add()([x, res])
    x = keras.layers.MaxPool2D()(x)

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name="improved_cnn")
    return model

model_imp = improved_model()
model_imp.summary()

In [ ]:
model_slim = keras.models.Sequential([
    keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(IMG_HEIGHT,IMG_WIDTH,channels),padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(43, activation='softmax')
], name="ultra_light_cnn")

model_slim.summary()

### MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

IMG_HEIGHT_MOBILENETV2 = 32
IMG_WIDTH_MOBILENETV2 = 32

def get_X_train_MobileNetV2():
    image_data = []
    image_labels = []
    
    for i in range(NUM_CATEGORIES):
        path = data_dir + '/train/' + f"{i:02d}"
    
        # Skip folder of label that not exists 
        if not os.path.isdir(path):
            continue
        images = os.listdir(path)
    
        for img in images:
            try:
                image = cv2.imread(path + '/' + img)
                image_fromarray = Image.fromarray(image, 'RGB')
                resize_image = image_fromarray.resize((IMG_HEIGHT_MOBILENETV2, IMG_WIDTH_MOBILENETV2))
                image_data.append(np.array(resize_image))
                image_labels.append(f"{i:02d}")
            except:
                print("Error in " + img)
    
    # Changing the list to numpy array
    image_data = np.array(image_data)
    image_labels = np.array(image_labels)
    return image_data.astype('float32') / 255. , image_labels

def get_X_val_MobileNetV2():
    val_data = []
    val_labels = []
    for label_name in os.listdir(val_set):
        label_dir = os.path.join(val_set, label_name)
        if not os.path.isdir(label_dir):
            continue
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = cv2.imread(img_path)
            img = Image.fromarray(img, 'RGB').resize((IMG_HEIGHT_MOBILENETV2, IMG_WIDTH_MOBILENETV2))
            val_data.append(np.array(img))
            val_labels.append(int(label_name))
    
    X_val = np.array(val_data, dtype='float32') / 255.
    return X_val, val_labels

X_train_MobileNetV2, image_labels_MobileNetV2 = get_X_train_MobileNetV2()
X_val_MobileNetV2, val_labels_MobileNetV2 = get_X_val_MobileNetV2()

y_train_MobileNetV2 = keras.utils.to_categorical(image_labels_MobileNetV2, NUM_CATEGORIES)
val_labels_MobileNetV2 = np.array(val_labels) 
y_val_MobileNetV2 = keras.utils.to_categorical(val_labels_MobileNetV2, NUM_CATEGORIES)

base = MobileNetV2(input_shape=(IMG_HEIGHT_MOBILENETV2,IMG_WIDTH_MOBILENETV2,channels), include_top=False, weights=None)
x = base.output
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(43, activation='softmax')(x)
model_mnv2 = models.Model(inputs=base.input, outputs=outputs, name="mobilenet_v2")

model_mnv2.summary()

## Augmenting the data and training the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler 
from keras.callbacks import ModelCheckpoint

# Data augmentation
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest"
)
augmented_train = aug.flow(X_train, y_train, batch_size=32)

# Monitor learning by validation accuracy
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,                
    restore_best_weights=True,
    verbose=1                 
)

checkpoint = ModelCheckpoint('model.weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

def lr_decay(epoch):
    return lr * (0.5 ** (epoch // (epochs * 0.5)))

In [ ]:
if os.path.isfile('/kaggle/input/weights/model.weights.h5'):
    model.load_weights('/kaggle/input/weights/model.weights.h5')
else:
    opt = Adam(learning_rate=lr)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    
    history_model = model.fit(
        augmented_train,
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=[early_stop, LearningRateScheduler(lr_decay), checkpoint]
    )
    model.save_weights('model.weights.h5')

In [ ]:
if os.path.isfile('/kaggle/input/weights/model_imp.weights.h5'):
    model.load_weights('/kaggle/input/weights/model_imp.weights.h5')
else:
    opt = Adam(learning_rate=lr)
    model_imp.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )

    history_model_imp = model_imp.fit(
        augmented_train,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[early_stop, LearningRateScheduler(lr_decay), checkpoint]
    )
    model_imp.save_weights('model_imp.weights.h5')

In [ ]:
if os.path.isfile('/kaggle/input/weights/model_slim.weights.h5'):
    model.load_weights('/kaggle/input/weights/model_slim.weights.h5')
else:
    opt = Adam(learning_rate=lr)
    model_slim.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    
    history_model_slim = model_slim.fit(
        augmented_train,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[early_stop,LearningRateScheduler(lr_decay), checkpoint]
    )
    model_slim.save_weights('model_slim.weights.h5')

In [ ]:
opt = Adam(learning_rate=lr)

model_mnv2.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Training con EarlyStopping
history_model_mnv2 = model_mnv2.fit(
    aug.flow(X_train_MobileNetV2, y_train_MobileNetV2, batch_size=32),
    epochs=30,
    validation_data=(X_val_MobileNetV2, y_val_MobileNetV2),
    callbacks=[LearningRateScheduler(lr_decay), checkpoint]
)
model_mnv2.save_weights('model_mnv2.weights.h5')

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(24, 5))

#histories = [history_model, history_model_imp, history_model_slim, history_model_mnv2]
histories = [history_model, history_model_imp, history_model_slim]
#models = [model, model_imp, model_slim, model_mnv2]
models = [model, model_imp, model_slim]
#titles = ['Training original model', 'Training improved model', 'Training slim model', 'MobileNetV2']
titles = ['Training original model', 'Training improved model', 'Training slim model']

for i, (hist, model, title) in enumerate(zip(histories, models, titles)):
    pd.DataFrame(hist.history).plot(ax=axs[i])
    axs[i].set_title(f"{title}\nParams: {model.count_params():,}")
    axs[i].grid(True)
    axs[i].set_ylim(0, 1)

plt.tight_layout()
plt.show()

## Evaluating the model

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

# Quantization Optimitation 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # default to 8bit
quant_model = converter.convert()

_, quant_model_file = tempfile.mkstemp('.tflite')

with open(quant_model_file, 'wb') as f:
  f.write(quant_model)

print('Saved quant TFLite model to:', quant_model_file)

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model:\t%.2f bytes" % (get_gzipped_model_size(model_file)))
print("Size of gzipped quant Keras model:\t%.2f bytes" % (get_gzipped_model_size(quant_model_file)))

## Loading the test data and running the predictions

In [ ]:
import glob

imgs = []
labels = []

for label_name in os.listdir(test_path):
    label_dir = os.path.join(test_path, label_name)
    if os.path.isdir(label_dir):
        # Cerca immagini dentro la cartella della label
        for img_path in glob.glob(os.path.join(label_dir, '*')):
            imgs.append(img_path)
            labels.append(label_name)

labels = np.array(labels).astype(int)

In [ ]:
def get_predictions(model):
    data = []

    for img in imgs:
        image = cv2.imread(img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    X_test = np.array(data)
    X_test = X_test/255
    
    pred = np.argmax(model.predict(X_test), axis=-1)
    return pred, X_test

In [ ]:
#Accuracy with the test data
print('Test Data accuracy original model: ',accuracy_score(labels, get_predictions(model)[0])*100)
print('Test Data accuracy improved model: ',accuracy_score(labels, get_predictions(model_imp)[0])*100)
print('Test Data accuracy improved model: ',accuracy_score(labels, get_predictions(model_slim)[0])*100)

# Weight Clustering

In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model_imp, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()


In [ ]:
# Fine-tune model
clustered_model.fit(
  aug.flow(X_train, y_train, batch_size=500),
  epochs=1,
  validation_data=(X_val, y_val)
)

## Visualizing the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(labels,  get_predictions(model_imp))

In [ ]:
import seaborn as sns
df_cm = pd.DataFrame(cf, index = classes,  columns = classes)
plt.figure(figsize = (20,20))
sns.heatmap(df_cm, annot=True)

## Classification report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels, get_predictions(model)[0]))
print(classification_report(labels, get_predictions(model_imp)[0]))
print(classification_report(labels, get_predictions(model_slim)[0]))

## Predictions on Test Data

In [ ]:
plt.figure(figsize = (25, 25))

pred, X_test = get_predictions(model_imp)

start_index = 0
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = pred[start_index + i]
    actual = labels[start_index + i]
    col = 'g'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col)
    plt.imshow(X_test[start_index + i])
plt.show()

# Lime Explainability Evaluation

In [ ]:
def predict_fn_1(images_np):
    images_np = images_np / 255.0
    return model.predict(images_np)

def predict_fn_2(images_np):
    images_np = images_np / 255.0
    return model_imp.predict(images_np)

def predict_fn_3(images_np):
    images_np = images_np / 255.0
    return model_slim.predict(images_np)

models = [
    ("Modello di base", predict_fn_1),
    ("Modello ottimizzato", predict_fn_2),
    ("Modello slim", predict_fn_3)
]

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/kaggle/input/dataset-2025-04-23-eps-100-changed/dataset_20250423_200322_eps_100_changed/test/22/132836142802631_29.jpg"
img = image.load_img(img_path)
img_np = image.img_to_array(img)
img_np = np.uint8(img_np)

In [ ]:
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

def show_lime_explanation(image, explainer, predict_fn, model_name):
    """
    Genera una spiegazione LIME per un singolo modello.
    Restituisce l'immagine processata, la heatmap e il nome del modello.
    """
    explanation = explainer.explain_instance(image, predict_fn, top_labels=5, hide_color=0, num_samples=1000)
    top_label = explanation.top_labels[0]
    weights = dict(explanation.local_exp[top_label])
    
    temp, mask = explanation.get_image_and_mask(
        label=top_label,
        positive_only=False,
        num_features=15,
        hide_rest=False
    )
    
    # Creiamo una heatmap più dettagliata
    heatmap = np.zeros_like(mask, dtype=float)
    for seg_idx, weight in weights.items():
        if abs(weight) > 0.005:  # Soglia più bassa per catturare più dettagli
            heatmap[mask == seg_idx] = weight
    
    # Normalizzazione simmetrica per preservare segno e intensità
    max_abs_weight = np.max(np.abs(heatmap))
    if max_abs_weight > 0:
        heatmap = heatmap / max_abs_weight
    
    # Applicazione di un filtro gaussiano per smoothing graduato
    heatmap_smooth = gaussian_filter(heatmap, sigma=1.5)
    
    return temp, heatmap_smooth, model_name

In [ ]:
lime_results = []
for i, (name, predict_fn) in enumerate(models):
    print(f"Processando {name}... ({i+1}/3)")
    temp, heatmap, model_name = show_lime_explanation(img_np, explainer, predict_fn, name)
    lime_results.append((temp, heatmap, model_name))

# Estrazione delle heatmap per la normalizzazione comune
heatmaps = [result[1] for result in lime_results]

# Normalizziamo tutte le heatmap sulla stessa scala per confronto
all_heatmaps = np.concatenate([h.flatten() for h in heatmaps])
vmin, vmax = np.percentile(all_heatmaps, [5, 95])  # Usa percentili per evitare outlier

# Visualizzazione migliorata
fig, axes = plt.subplots(1, 4, figsize=(20, 6))

# Immagine originale
axes[0].imshow(img_np.astype(np.uint8))
axes[0].set_title("Immagine originale", fontsize=12, fontweight='bold')
axes[0].axis('off')

# Visualizzazione delle heatmap usando i risultati pre-calcolati
for i, (temp, heatmap, model_name) in enumerate(lime_results):
    # Mostra l'immagine originale
    axes[i + 1].imshow(temp.astype(np.uint8))
    
    # Overlay della heatmap con colormap migliorata
    im = axes[i + 1].imshow(heatmap, 
                           cmap='RdYlBu_r',  # Colormap che va dal blu (negativo) al rosso (positivo)
                           alpha=0.6,        # Trasparenza per vedere l'immagine sotto
                           vmin=vmin, 
                           vmax=vmax,
                           interpolation='bilinear')  # Interpolazione per smoothness
    
    axes[i + 1].set_title(f"{model_name}", fontsize=12, fontweight='bold')
    axes[i + 1].axis('off')

# Colorbar migliorata
fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label("Importanza\n(Rosso=Positiva, Blu=Negativa)", fontsize=10, fontweight='bold')
cbar.ax.tick_params(labelsize=9)

# Layout finale
plt.tight_layout()
plt.subplots_adjust(right=0.88)
plt.show()

## Analisi testuali risultati spiegabilità

In [ ]:
print("\n" + "="*80)
print("ANALISI DETTAGLIATA DELLE SPIEGAZIONI LIME")
print("="*80)

def analyze_lime_result(heatmap, model_name):
    """
    Analizza una heatmap LIME e restituisce statistiche descrittive.
    """
    # Calcolo statistiche di base
    positive_weights = heatmap[heatmap > 0]
    negative_weights = heatmap[heatmap < 0]
    
    total_pixels = heatmap.size
    positive_pixels = len(positive_weights)
    negative_pixels = len(negative_weights)
    neutral_pixels = total_pixels - positive_pixels - negative_pixels
    
    # Percentuali
    pos_percentage = (positive_pixels / total_pixels) * 100
    neg_percentage = (negative_pixels / total_pixels) * 100
    neutral_percentage = (neutral_pixels / total_pixels) * 100
    
    # Statistiche sui pesi
    max_positive = np.max(positive_weights) if len(positive_weights) > 0 else 0
    max_negative = np.min(negative_weights) if len(negative_weights) > 0 else 0
    mean_positive = np.mean(positive_weights) if len(positive_weights) > 0 else 0
    mean_negative = np.mean(negative_weights) if len(negative_weights) > 0 else 0
    
    # Intensità complessiva
    total_positive_influence = np.sum(positive_weights) if len(positive_weights) > 0 else 0
    total_negative_influence = np.sum(np.abs(negative_weights)) if len(negative_weights) > 0 else 0
    
    return {
        'positive_pixels': positive_pixels,
        'negative_pixels': negative_pixels,
        'neutral_pixels': neutral_pixels,
        'pos_percentage': pos_percentage,
        'neg_percentage': neg_percentage,
        'neutral_percentage': neutral_percentage,
        'max_positive': max_positive,
        'max_negative': max_negative,
        'mean_positive': mean_positive,
        'mean_negative': mean_negative,
        'total_positive_influence': total_positive_influence,
        'total_negative_influence': total_negative_influence
    }

def interpret_lime_behavior(stats, model_name):
    """
    Interpreta le statistiche LIME e fornisce una descrizione comportamentale.
    """
    interpretation = []
    
    # Analisi della distribuzione delle influenze
    if stats['pos_percentage'] > stats['neg_percentage']:
        if stats['pos_percentage'] > 60:
            interpretation.append("Il modello si basa prevalentemente su caratteristiche positive dell'immagine per la sua predizione.")
        else:
            interpretation.append("Il modello utilizza principalmente caratteristiche positive, ma considera anche alcuni aspetti negativi.")
    elif stats['neg_percentage'] > stats['pos_percentage']:
        if stats['neg_percentage'] > 60:
            interpretation.append("Il modello sembra essere principalmente influenzato da caratteristiche che considera controindizi.")
        else:
            interpretation.append("Il modello bilancia caratteristiche positive e negative, con una leggera prevalenza dei controindizi.")
    else:
        interpretation.append("Il modello presenta un bilanciamento equilibrato tra caratteristiche positive e negative.")
    
    # Analisi dell'intensità
    if stats['max_positive'] > 0.8:
        interpretation.append("Presenza di aree ad alta confidenza positiva, indicando caratteristiche molto distintive per la classe predetta.")
    elif stats['max_positive'] > 0.5:
        interpretation.append("Il modello identifica alcune caratteristiche moderatamente importanti per la predizione.")
    else:
        interpretation.append("Il modello mostra una confidenza relativamente bassa nelle caratteristiche individuate.")
    
    if abs(stats['max_negative']) > 0.8:
        interpretation.append("Presenza di forti controindizi, suggerendo caratteristiche che il modello associa ad altre classi.")
    elif abs(stats['max_negative']) > 0.5:
        interpretation.append("Il modello identifica alcuni controindizi moderati nella predizione.")
    
    # Analisi della selettività
    if stats['neutral_percentage'] > 70:
        interpretation.append("Il modello è molto selettivo, concentrandosi solo su specifiche aree dell'immagine.")
    elif stats['neutral_percentage'] > 50:
        interpretation.append("Il modello mostra una selettività moderata nell'analisi dell'immagine.")
    else:
        interpretation.append("Il modello considera una vasta porzione dell'immagine nella sua decisione.")
    
    # Analisi dell'influenza totale
    total_influence = stats['total_positive_influence'] + stats['total_negative_influence']
    if total_influence > stats['positive_pixels'] * 0.6:  # Soglia arbitraria
        interpretation.append("Il modello mostra una forte reattività complessiva alle caratteristiche dell'immagine.")
    else:
        interpretation.append("Il modello presenta una reattività moderata alle caratteristiche visive.")
    
    return interpretation

# Analisi per ogni modello
for i, (temp, heatmap, model_name) in enumerate(lime_results):
    print(f"\n🔍 MODELLO: {model_name.upper()}")
    print("-" * 50)
    
    # Calcolo statistiche
    stats = analyze_lime_result(heatmap, model_name)
    
    # Stampa statistiche numeriche
    print(f"📊 STATISTICHE NUMERICHE:")
    print(f"   • Pixel con influenza positiva: {stats['positive_pixels']:,} ({stats['pos_percentage']:.1f}%)")
    print(f"   • Pixel con influenza negativa: {stats['negative_pixels']:,} ({stats['neg_percentage']:.1f}%)")
    print(f"   • Pixel neutrali: {stats['neutral_pixels']:,} ({stats['neutral_percentage']:.1f}%)")
    print(f"   • Massima influenza positiva: {stats['max_positive']:.3f}")
    print(f"   • Massima influenza negativa: {stats['max_negative']:.3f}")
    print(f"   • Influenza positiva media: {stats['mean_positive']:.3f}")
    print(f"   • Influenza negativa media: {stats['mean_negative']:.3f}")
    
    # Interpretazione comportamentale
    interpretations = interpret_lime_behavior(stats, model_name)
    print(f"\n🧠 INTERPRETAZIONE COMPORTAMENTALE:")
    for j, interp in enumerate(interpretations, 1):
        print(f"   {j}. {interp}")
    
    # Confronto relativo (solo dal secondo modello in poi)
    if i > 0:
        prev_stats = analyze_lime_result(lime_results[i-1][1], lime_results[i-1][2])
        print(f"\n🔄 CONFRONTO CON {lime_results[i-1][2].upper()}:")
        
        pos_diff = stats['pos_percentage'] - prev_stats['pos_percentage']
        if abs(pos_diff) > 5:  # Differenza significativa
            direction = "più" if pos_diff > 0 else "meno"
            print(f"   • Questo modello è {direction} orientato verso caratteristiche positive ({pos_diff:+.1f}%)")
        
        max_pos_diff = stats['max_positive'] - prev_stats['max_positive']
        if abs(max_pos_diff) > 0.1:
            direction = "maggiore" if max_pos_diff > 0 else "minore"
            print(f"   • Mostra una confidenza {direction} nelle caratteristiche chiave ({max_pos_diff:+.3f})")
        
        neutral_diff = stats['neutral_percentage'] - prev_stats['neutral_percentage']
        if abs(neutral_diff) > 10:
            direction = "più" if neutral_diff > 0 else "meno"
            print(f"   • È {direction} selettivo nell'analisi dell'immagine ({neutral_diff:+.1f}%)")

print(f"\n" + "="*80)
print("RIEPILOGO COMPARATIVO")
print("="*80)

# Riepilogo finale comparativo
best_positive = max(lime_results, key=lambda x: analyze_lime_result(x[1], x[2])['max_positive'])
most_selective = max(lime_results, key=lambda x: analyze_lime_result(x[1], x[2])['neutral_percentage'])
most_balanced = min(lime_results, key=lambda x: abs(analyze_lime_result(x[1], x[2])['pos_percentage'] - analyze_lime_result(x[1], x[2])['neg_percentage']))

print(f"🏆 Modello con maggiore confidenza: {best_positive[2]}")
print(f"🎯 Modello più selettivo: {most_selective[2]}")
print(f"⚖️  Modello più bilanciato: {most_balanced[2]}")
print("="*80)

# Studio disomogeneità geografiche

In [ ]:
def get_predictions_for_area_study(model, image_paths, img_size=(IMG_HEIGHT, IMG_WIDTH)):
    data = []

    for img_path in image_paths:
        try:
            img = cv2.imread(img_path)  # BGR
            img_pil = Image.fromarray(img, mode='RGB')
            img_resized = img_pil.resize(img_size)
            data.append(np.array(img_resized))
        except Exception as e:
            print(f"Errore con immagine {img_path}: {e}")
            data.append(np.zeros((*img_size, 3)))  # Fallback

    X = np.array(data, dtype='float32') / 255.0
    preds = np.argmax(model.predict(X), axis=-1)
    
    return preds

def geo_label(lat):
    if lat < 41.5594700:
        return 'sud'
    elif lat > 44.801485:
        return 'nord'
    else:
        return 'centre'

def build_paths(df, base_path='./'):
    return [
        os.path.join(base_path, 'test', f"{row['feature_index']:02d}", row['filename'])
        for _, row in df.iterrows()
    ]

model_dict = {
    'Model': model,
    'Model_Imp': model_imp,
    'Model_Slim': model_slim
}

In [ ]:
sign_info = pd.read_parquet(f'{data_dir}/metadata.parquet')
sign_info = sign_info.loc[sign_info['split'] == 'test']

# Da rimuovere
sign_info[['lat', 'lon']] = sign_info['coordinates'].str.split(",", expand=True).astype(float)
sign_info['geo_area'] = sign_info['lat'].apply(geo_label)

sign_info_nord = sign_info[sign_info['geo_area'] == 'nord']
sign_info_centre = sign_info[sign_info['geo_area'] == 'centre']
sign_info_sud = sign_info[sign_info['geo_area'] == 'sud']

area_dfs = {'nord': sign_info_nord, 'centre': sign_info_centre, 'sud': sign_info_sud}
accuracy_results = []

In [ ]:
for model_name, mdl in model_dict.items():
    for area_name, df_area in area_dfs.items():
        image_paths = build_paths(df_area, data_dir)
        preds = get_predictions_for_area_study(mdl, image_paths)
        true_labels = df_area['feature_index'].values
        acc = np.mean(preds == true_labels)
        accuracy_results.append({
            'modello': model_name,
            'area': area_name,
            'accuracy': acc
        })

results_df = pd.DataFrame(accuracy_results)

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 5))
heatmap_data = results_df.pivot(index='modello', columns='area', values='accuracy')

sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt=".2f", cbar=True)
plt.title("Accuracy per area geografica per modello")
plt.xlabel("Area")
plt.ylabel("Modello")
plt.tight_layout()
plt.show()